In [1]:
from typing import Optional, List
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# Initializing Spark

In [2]:
# sc & ss
AppName, Mode = "Python Spark Example", "local"
conf = SparkConf().setAppName(AppName).setMaster(Mode)
sc = SparkContext(conf=conf)
ss = SparkSession.builder.getOrCreate()
print(f"SparkContext: {sc}")
print(f"SparkSession: {ss}")

SparkContext: <SparkContext master=local appName=Python Spark Example>
SparkSession: <pyspark.sql.session.SparkSession object at 0xffff65649210>


# RDD

## Create RDD

In [3]:
# with Parallelized Collections
data = [
    [1, 2, 3], 
    [4, 5],
    [6],
    [7, 8, 9, 10]
]
persist_data = sc.parallelize(data).persist()
print(f"Type: {type(persist_data)}, Data: {persist_data.collect()}")

Type: <class 'pyspark.rdd.RDD'>, Data: [[1, 2, 3], [4, 5], [6], [7, 8, 9, 10]]


## RDD Operations

In [4]:
# Basic Operation 
print(f" first: {persist_data.first()}")
print(f" take: {persist_data.take(2)}")
print(f" count: {persist_data.count()}")
print(f" collect: {persist_data.collect()}")
print(f" countByKey: {persist_data.countByKey()}")

 first: [1, 2, 3]
 take: [[1, 2, 3], [4, 5]]
 count: 4
 collect: [[1, 2, 3], [4, 5], [6], [7, 8, 9, 10]]
 countByKey: defaultdict(<class 'int'>, {1: 1, 4: 1, 6: 1, 7: 1})


In [5]:
# Map
result = persist_data.map(lambda x: [i+1 for i in x])
print(f" Result: {result.collect()}")

 Result: [[2, 3, 4], [5, 6], [7], [8, 9, 10, 11]]


In [6]:
# flatMap
result = persist_data.flatMap(lambda x: [i+1 for i in x])
print(f" Result: {result.collect()}")

 Result: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [7]:
# filter
result = persist_data.filter(lambda x: 7 in x)
print(f" Result: {result.collect()}")

 Result: [[7, 8, 9, 10]]


In [8]:
# foreach: Run a function func on each element of the dataset. 
# This is usually done for side effects such as updating an Accumulator or interacting with external storage systems. 
accum = sc.accumulator(0)

class MyAccumulator:
    def AccumulateByLength(nums: List[int]) -> None:
        target = len(nums)
        accum.add(target)
    def AccumulateByFirstNum(nums: List[int]) -> None:
        target = nums[0]
        accum.add(target)

persist_data.foreach(MyAccumulator.AccumulateByLength)  # 3+2+1+4 = 10
print(f" Result_1: {accum.value}") #Accessed by driver

persist_data.foreach(MyAccumulator.AccumulateByFirstNum) # 10+ (1+4+6+7) = 28
print(f" Result_2: {accum.value}") #Accessed by driver

 Result_1: 10
 Result_2: 28


In [9]:
# map: Return a new distributed dataset formed by passing each element of the source through a function func.
# Passing Functions to Spark

class MyComputer:
    def plusone(nums: List[int]) -> list:
        return [i+1 for i in nums]
    def minusone(nums: List[int]) -> list:
        return [i-1 for i in nums]

result_1 = persist_data.map(MyComputer.plusone)
result_2 = persist_data.map(MyComputer.minusone)
print(f" Result_1: {result_1.collect()}")
print(f" Result_2: {result_2.collect()}")

 Result_1: [[2, 3, 4], [5, 6], [7], [8, 9, 10, 11]]
 Result_2: [[0, 1, 2], [3, 4], [5], [6, 7, 8, 9]]


# Spark DataFrame

## Create DataFrame

In [134]:
# prepare json data
people_json = [
    {"Name": "Jacky", "Age": 27, "Gender": "Male", "Country": "Taiwan"},
    {"Name": "John", "Age": 32, "Gender": "Male", "Country": "Taiwan"},
    {"Name": "Tom", "Age": 42, "Gender": "Male", "Country": "Japan"},
    {"Name": "Mark", "Age": 16, "Gender": "Male", "Country": "Taiwan"},
    {"Name": "Sandy", "Age": 23, "Gender": "Female", "Country": "US"}
]

In [135]:
# Use rdd.toDF() to create a spark DataFrame
people_json = sc.parallelize(people_json)
sample_df = persist_people_json.toDF()
sample_df.printSchema()
sample_df.show()

root
 |-- Age: long (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Name: string (nullable = true)

+---+-------+------+-----+
|Age|Country|Gender| Name|
+---+-------+------+-----+
| 27| Taiwan|  Male|Jacky|
| 32| Taiwan|  Male| John|
| 42|  Japan|  Male|  Tom|
| 16| Taiwan|  Male| Mark|
| 23|     US|Female|Sandy|
+---+-------+------+-----+



In [136]:
# Create DataFrame with a specified schema
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
deptSchema = StructType([       
    StructField('Name', StringType(), False),
    StructField('Age', IntegerType(), True),
    StructField('Gender', StringType(), True),
    StructField('Country', StringType(), True)
])

df = ss.createDataFrame(people_json, schema = deptSchema)
df.printSchema()
df.show(truncate=False)

root
 |-- Name: string (nullable = false)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)

+-----+---+------+-------+
|Name |Age|Gender|Country|
+-----+---+------+-------+
|Jacky|27 |Male  |Taiwan |
|John |32 |Male  |Taiwan |
|Tom  |42 |Male  |Japan  |
|Mark |16 |Male  |Taiwan |
|Sandy|23 |Female|US     |
+-----+---+------+-------+



## DataFrame Operations

In [137]:
# Select people from Taiwan and older than 25
df.filter((df['Country'] == "Taiwan") & (df['Age'] > 25)) \
    .select(df['Name'], df['Age'], df['Country']) \
    .show()

+-----+---+-------+
| Name|Age|Country|
+-----+---+-------+
|Jacky| 27| Taiwan|
| John| 32| Taiwan|
+-----+---+-------+



In [138]:
# groupBy & where with functions
from pyspark.sql.functions import col, sum, avg, max, min
df.groupBy("Country") \
  .agg(
    count("Country").alias("Count_Country"),
    min("Age").alias("Min_Age"),
    max("Age").alias("Max_Age"),
    sum("Age").alias("Sum_Age"),
    avg("Age").alias("Average_Age")
  ) \
  .where(col("Average_Age") >= 10) \
  .show()

+-------+-------------+-------+-------+-------+-----------+
|Country|Count_Country|Min_Age|Max_Age|Sum_Age|Average_Age|
+-------+-------------+-------+-------+-------+-----------+
| Taiwan|            3|     16|     32|     75|       25.0|
|     US|            1|     23|     23|     23|       23.0|
|  Japan|            1|     42|     42|     42|       42.0|
+-------+-------------+-------+-------+-------+-----------+



In [139]:
# UDF
from pyspark.sql.functions import udf

class MyUdf:
    def newname(s: str) -> str:
        return f"NEW_{s}"
    def powerage(i: int) -> int:
        return i*i

# Convert function to udf
newname_udf = udf(lambda x: MyUdf.newname(x), StringType())
powerage_udf = udf(lambda x: MyUdf.powerage(x), IntegerType())

# Apply udf
df1 = df.withColumn("New Name", newname_udf(col("Name")))
df1 = df1.withColumn("Power Age", powerage_udf(col("Age")))
df1.show()

+-----+---+------+-------+---------+---------+
| Name|Age|Gender|Country| New Name|Power Age|
+-----+---+------+-------+---------+---------+
|Jacky| 27|  Male| Taiwan|NEW_Jacky|      729|
| John| 32|  Male| Taiwan| NEW_John|     1024|
|  Tom| 42|  Male|  Japan|  NEW_Tom|     1764|
| Mark| 16|  Male| Taiwan| NEW_Mark|      256|
|Sandy| 23|Female|     US|NEW_Sandy|      529|
+-----+---+------+-------+---------+---------+



In [161]:
# PySpark DataFrame doesn’t contain the apply() function 
# However, we can leverage Pandas DataFrame.apply() by running Pandas API over PySpark.
import pyspark.pandas as ps

psdf = ps.DataFrame(df)
print(f"Type: {type(psdf)} \n {psdf.head(1)}")

# Apply function to psdf
psdf["New Name"] = psdf["Name"].apply(MyUdf.newname)
psdf["Power Age"] = psdf["Age"].apply(MyUdf.powerage)
psdf.head()

Type: <class 'pyspark.pandas.frame.DataFrame'> 
     Name  Age Gender Country
0  Jacky   27   Male  Taiwan


,Name,Age,Gender,Country,New Name,Power Age
0,Jacky,27,Male,Taiwan,NEW_Jacky,729
1,John,32,Male,Taiwan,NEW_John,1024
2,Tom,42,Male,Japan,NEW_Tom,1764
3,Mark,16,Male,Taiwan,NEW_Mark,256
4,Sandy,23,Female,US,NEW_Sandy,529


# Spark SQL

## Create TempView

In [170]:
df.createOrReplaceTempView("PEOPLE_TABLE")
df.createOrReplaceGlobalTempView("GLOBAL_PEOPLE_TABLE")

## SQL Operations

In [171]:
# select from table / global table
ss.sql("SELECT * FROM PEOPLE_TABLE").show()
ss.sql("SELECT * FROM global_temp.GLOBAL_PEOPLE_TABLE").show()

+-----+---+------+-------+
| Name|Age|Gender|Country|
+-----+---+------+-------+
|Jacky| 27|  Male| Taiwan|
| John| 32|  Male| Taiwan|
|  Tom| 42|  Male|  Japan|
| Mark| 16|  Male| Taiwan|
|Sandy| 23|Female|     US|
+-----+---+------+-------+

+-----+---+------+-------+
| Name|Age|Gender|Country|
+-----+---+------+-------+
|Jacky| 27|  Male| Taiwan|
| John| 32|  Male| Taiwan|
|  Tom| 42|  Male|  Japan|
| Mark| 16|  Male| Taiwan|
|Sandy| 23|Female|     US|
+-----+---+------+-------+



In [177]:
# sql operation
ss.sql("""
        SELECT
           Name, Upper(Name) AS NEW_Name
        FROM
           PEOPLE_TABLE
       """
      ).show()

+-----+--------+
| Name|NEW_Name|
+-----+--------+
|Jacky|   JACKY|
| John|    JOHN|
|  Tom|     TOM|
| Mark|    MARK|
|Sandy|   SANDY|
+-----+--------+



In [181]:
# sql with UDF
class MySqlUdf:
    def lowercase(s: str) -> str:
        return s.lower()
    def uppercase(s: str) -> str:
        return s.upper()
ss.udf.register("lowercase_udf", MySqlUdf.lowercase)
ss.udf.register("uppercase_udf", MySqlUdf.uppercase)

<function __main__.MySqlUdf.uppercase(s: str) -> str>

In [183]:
ss.sql("""
        SELECT
           Name, 
           lowercase_udf(Name) AS LOWER_Name,
           uppercase_udf(Name) AS UPPER_Name
        FROM
           PEOPLE_TABLE
       """
      ).show()

+-----+----------+----------+
| Name|LOWER_Name|UPPER_Name|
+-----+----------+----------+
|Jacky|     jacky|     JACKY|
| John|      john|      JOHN|
|  Tom|       tom|       TOM|
| Mark|      mark|      MARK|
|Sandy|     sandy|     SANDY|
+-----+----------+----------+

